<a href="https://colab.research.google.com/github/ambideXtrous9/Finetune-Gemma2b-for-QA-RAG-LoRA-ChromaDB/blob/main/flant5_finetune_rag_with_faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y pyarrow
!pip install -qq -U pyarrow==15.0.2
!pip install -qq -U datasets
!pip install -qq -U transformers
!pip install -qq -U langchain-community
!pip install -qq -U sentence_transformers==2.2.2
!pip install -qq -U faiss-gpu

Found existing installation: pyarrow 14.0.2
Uninstalling pyarrow-14.0.2:
  Successfully uninstalled pyarrow-14.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 8.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50

In [2]:
import pandas as pd
from datasets import Dataset
import torch
from transformers import DataCollatorForSeq2Seq
from transformers import  Trainer, TrainingArguments
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import DataFrameLoader
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import TextStreamer,pipeline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/MTP CODE/NewsQA_SPAN.feather'

In [5]:
df = pd.read_feather(path)
df

,question,answer,ans_pos,paragraph,answer_start,answer_end
0,Who is the managing director of Synergee Capital?,Vikram Dalal,"[133, 145]","""Investors can use a combination of governmen...",133,145
1,What is the yield of 30- and 40-year governmen...,7%,"[565, 567]","""Investors can use a combination of governmen...",565,567
2,What is the name of the ETF 2027 that a conser...,SDL,"[209, 212]","According to financial planners, an example o...",209,212
3,When would a conservative fixed income investo...,2027,"[217, 221]","According to financial planners, an example o...",217,221
4,What year would a conservative fixed income in...,2040,"[260, 264]","According to financial planners, an example o...",260,264
...,...,...,...,...,...,...
481753,When does Uncle Sam reopen for fully vaccinate...,November 8,"[295, 305]",NEW DELHI: This could be the last expansion of...,295,305
481754,When will there be three more weekly flights b...,from second week of November,"[116, 144]",It currently has 23 weekly flights to America....,116,144
481755,What type of 777s would have helped AI have mo...,Boeing,"[306, 312]",It currently has 23 weekly flights to America....,306,312
481756,What was the first wave of AI nonstops?,second,"[11, 17]","Before the second wave this summer, AI had abo...",11,17


In [6]:
df = df[:5000]

In [7]:
# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [8]:
dataset = dataset.train_test_split(test_size=0.2)  # Adjust `test_size` as needed

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'ans_pos', 'paragraph', 'answer_start', 'answer_end', '__index_level_0__'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['question', 'answer', 'ans_pos', 'paragraph', 'answer_start', 'answer_end', '__index_level_0__'],
        num_rows: 1000
    })
})

In [10]:
model_name = "google-t5/t5-small"

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
def preprocess_function(examples):
    # Concatenate question and context

    template = """
    Use the following piece of context to answer the question in less than 30 words.

    Context: {context}

    Question: {question}

    Answer: """

    # Apply the template to create inputs by formatting the question and context
    inputs = [
        template.format(question=question, context=context)
        for question, context in zip(examples['question'], examples['paragraph'])
    ]

    # Tokenize the inputs
    tokenized_inputs = tokenizer(inputs,
                                 padding="max_length",
                                 truncation=True,
                                 return_tensors="pt",
                                 max_length=400)

    # Tokenize the answers as labels
    labels = tokenizer(examples['answer'],
                       padding="max_length",
                       truncation=True,
                       return_tensors="pt",
                       max_length=30)

    return {
        'input_ids': tokenized_inputs['input_ids'].squeeze(),
        'attention_mask': tokenized_inputs['attention_mask'].squeeze(),
        'labels': labels['input_ids'].squeeze()
    }

In [13]:
trained_dataset = dataset["train"].map(preprocess_function,remove_columns= ['question', 'answer', 'paragraph'], batch_size=64,batched=True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [14]:
validation_dataset = dataset["test"].map(preprocess_function,remove_columns= ['question', 'answer', 'paragraph'], batch_size=64,batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [15]:
trained_dataset

Dataset({
    features: ['ans_pos', 'answer_start', 'answer_end', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 4000
})

In [16]:
validation_dataset

Dataset({
    features: ['ans_pos', 'answer_start', 'answer_end', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)


In [18]:
EPOCHS = 20

In [19]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    overwrite_output_dir=True, # This reduces the amt of disk space that gets used.
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    num_train_epochs=EPOCHS,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit = 1,
    report_to="none",
    load_best_model_at_end=True
)


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=trained_dataset,
    eval_dataset=validation_dataset
)

In [21]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.072293
2,No log,0.043971
3,No log,0.038264
4,No log,0.035124
5,0.429700,0.032021
6,0.429700,0.030160
7,0.429700,0.028304
8,0.429700,0.027489
9,0.429700,0.025271
10,0.029500,0.024655


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2000, training_loss=0.12505418014526368, metrics={'train_runtime': 2209.57, 'train_samples_per_second': 36.206, 'train_steps_per_second': 0.905, 'total_flos': 8458862592000000.0, 'train_loss': 0.12505418014526368, 'epoch': 20.0})

In [22]:
!huggingface-cli login --token $"YOUR_TOKEN"
model.push_to_hub("NewsQAFinetunedFlanT5s")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ambideXtrous9/NewsQAFinetunedFlanT5s/commit/a1700d460521a159f616eb31ffacb2c100a7c510', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='a1700d460521a159f616eb31ffacb2c100a7c510', pr_url=None, pr_revision=None, pr_num=None)

In [25]:
loader = DataFrameLoader(df, page_content_column="paragraph")


text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                               chunk_overlap=20)

documents = loader.load_and_split(text_splitter=text_splitter)


model_name1 = "sentence-transformers/all-MiniLM-L6-v2"

model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name1, model_kwargs=model_kwargs)

vectordb = FAISS.from_documents(
        documents = documents,
        embedding = embeddings
    )

<ipython-input-25-e9f10eae724a>:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name=model_name1, model_kwargs=model_kwargs)


.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [24]:
streamer = TextStreamer(tokenizer,
                        skip_prompt = True,
                        skip_special_tokens = True)

pipe =  pipeline(task = 'text2text-generation',
                         model="ambideXtrous9/NewsQAFinetunedFlanT5s",
                         tokenizer = tokenizer,
                         temperature=0.5,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
                         top_p=0.25,  # select from top tokens whose probability add up to 15%
                         top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
                         max_new_tokens=10,  # mex number of tokens to generate in the output
                         repetition_penalty=1.1,  # without this output begins repeating
                         do_sample = True,
                         )

llm = HuggingFacePipeline(pipeline = pipe)

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
<ipython-input-24-a8254515ba83>:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline = pipe)


In [31]:
prompt_template = """
                Use following piece of context to answer the question in less than 30 words.

                Context : {context}

                Question : {question}

                Answer : """


PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["context", "question"]
)


retriever = vectordb.as_retriever(search_kwargs = {"k": 5})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever,
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [32]:
def predict(idx):
    question = df.iloc[idx]['question']
    print("\nQuestion : ",question)
    print("\nContext : ",df.iloc[idx]['paragraph'])
    print("\nActual : ",df.iloc[idx]['answer'])
    result = qa_chain(question)
    print("\nPredicted : ", result['result'])
    return result

In [33]:
predict(8)

<ipython-input-32-44ab25496931>:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain(question)
Token indices sequence length is longer than the specified maximum sequence length for this model (555 > 512). Running this sequence through the model will result in indexing errors



Question :  Who is receiving their first dose of the COVID-19 vaccine?

Context :   The number of Americans receiving their first dose of COVID-19 vaccine is down about 85% since peaking in mid-April, and will likely fail to meet President Joe Biden's goal to have delivered at least one shot to 70% of adults by July 4. Warning warranted "Based on the available data, a warning statement in the factsheets for both healthcare providers and vaccine recipients and caregivers would be warranted," FDA official Doran Fink said at the advisory committee meeting.

Actual :  Americans

Predicted :  Americans


{'query': 'Who is receiving their first dose of the COVID-19 vaccine?',
 'result': 'Americans',
 'source_documents': [Document(metadata={'question': 'Who is receiving their first dose of the COVID-19 vaccine?', 'answer': 'Americans', 'ans_pos': array([14, 23]), 'answer_start': 14, 'answer_end': 23}, page_content='The number of Americans receiving their first dose of COVID-19 vaccine is down about 85% since peaking in mid-April, and will likely fail to meet President Joe Biden\'s goal to have delivered at least one shot to 70% of adults by July 4. Warning warranted "Based on the available data, a warning statement in the factsheets for both healthcare providers and vaccine recipients and caregivers would be warranted," FDA official Doran Fink said at the advisory committee meeting.'),
  Document(metadata={'question': 'How many doses of the COVID-19 vaccine did President Joe Biden want to deliver by July 4?', 'answer': 'at least one', 'ans_pos': array([191, 203]), 'answer_start': 191, 'a

In [34]:
predict(10)


Question :  How many doses of the COVID-19 vaccine did President Joe Biden want to deliver by July 4?

Context :   The number of Americans receiving their first dose of COVID-19 vaccine is down about 85% since peaking in mid-April, and will likely fail to meet President Joe Biden's goal to have delivered at least one shot to 70% of adults by July 4. Warning warranted "Based on the available data, a warning statement in the factsheets for both healthcare providers and vaccine recipients and caregivers would be warranted," FDA official Doran Fink said at the advisory committee meeting.

Actual :  at least one

Predicted :  at least one


{'query': 'How many doses of the COVID-19 vaccine did President Joe Biden want to deliver by July 4?',
 'result': 'at least one',
 'source_documents': [Document(metadata={'question': 'When did the first dose of the COVID-19 vaccine peak?', 'answer': 'mid-April', 'ans_pos': array([106, 115]), 'answer_start': 106, 'answer_end': 115}, page_content='The number of Americans receiving their first dose of COVID-19 vaccine is down about 85% since peaking in mid-April, and will likely fail to meet President Joe Biden\'s goal to have delivered at least one shot to 70% of adults by July 4. Warning warranted "Based on the available data, a warning statement in the factsheets for both healthcare providers and vaccine recipients and caregivers would be warranted," FDA official Doran Fink said at the advisory committee meeting.'),
  Document(metadata={'question': 'Who is receiving their first dose of the COVID-19 vaccine?', 'answer': 'Americans', 'ans_pos': array([14, 23]), 'answer_start': 14, 'answe

In [35]:
predict(22)


Question :  According to what agency, 138 million Americans have been fully vaccinated with one of the two mRNA vaccines?

Context :   Moderna said it is aware of reports of heart inflammation cases following administration of mRNA vaccines. It said it is working with public health and regulatory authorities to assess the issue. Over 138 million Americans have so far been fully vaccinated with one of the two mRNA vaccines, according to CDC data as of Monday.

Actual :  CDC

Predicted :  CDC


{'query': 'According to what agency, 138 million Americans have been fully vaccinated with one of the two mRNA vaccines?',
 'result': 'CDC',
 'source_documents': [Document(metadata={'question': 'On what date did the CDC release data on the mRNA vaccine?', 'answer': 'Monday', 'ans_pos': array([320, 326]), 'answer_start': 320, 'answer_end': 326}, page_content='Moderna said it is aware of reports of heart inflammation cases following administration of mRNA vaccines. It said it is working with public health and regulatory authorities to assess the issue. Over 138 million Americans have so far been fully vaccinated with one of the two mRNA vaccines, according to CDC data as of Monday.'),
  Document(metadata={'question': 'According to what agency, 138 million Americans have been fully vaccinated with one of the two mRNA vaccines?', 'answer': 'CDC', 'ans_pos': array([305, 308]), 'answer_start': 305, 'answer_end': 308}, page_content='Moderna said it is aware of reports of heart inflammation ca

In [36]:
predict(34)


Question :  Who did Blinken say would act with even greater impunity if they did not?

Context :   Secretary of State Antony Blinken said on Wednesday that the United States was ready to confront China where need be, calling the Asian power the “biggest geopolitical test” of the century. In his first major speech, Blinken vowed that President Joe Biden’s administration will emphasise diplomacy over military action and build cooperation with the world on global challenges such as climate change and Covid-19. “We will manage the biggest geopolitical test of the 21st century: our relationship with China,” Blinken said at the State Department. He promised to champion the rights of Hong Kong and the ethnic Uighurs, saying that if not, “China will act with even greater impunity”. “China is the only country with the economic, diplomatic, military and technological power to seriously challenge the stable and open international system — all the rules, values and relationships that make the wor

{'query': 'Who did Blinken say would act with even greater impunity if they did not?',
 'result': 'China',
 'source_documents': [Document(metadata={'question': 'Who ordered an air strike in Syria against Iranian-linked Iraqi Shiite paramilitaries?', 'answer': 'Biden', 'ans_pos': array([1149, 1154]), 'answer_start': 1149, 'answer_end': 1154}, page_content='Secretary of State Antony Blinken said on Wednesday that the United States was ready to confront China where need be, calling the Asian power the “biggest geopolitical test” of the century. In his first major speech, Blinken vowed that President Joe Biden’s administration will emphasise diplomacy over military action and build cooperation with the world on global challenges such as climate change and Covid-19. “We will manage the biggest geopolitical test of the 21st century: our relationship with China,” Blinken said at the State Department. He promised to champion the rights of Hong Kong and the ethnic Uighurs, saying that if not, “

In [ ]:
import shutil
shutil.make_archive('/kaggle/working/FinetunedFlanT5', 'zip', '/kaggle/working/my_awesome_model/checkpoint-2000')
